In [1]:
import numpy as np
import pandas as pd

In [2]:
ohio_2012_path = 'data/ohio 2012precinct.xlsx'
excel_2012 = pd.ExcelFile(ohio_2012_path)
sheet_names = excel_2012.sheet_names

In [3]:
for i in sheet_names:
    print(i)

Contents
Master
President
U.S. Congress
General Assembly
Judicial


In [4]:
data = pd.read_excel(ohio_2012_path, sheet_name='U.S. Congress', header=1)

In [5]:
data.columns #useful stuff here, but not for our purposes

Index(['County Name', 'Precinct Name', 'Precinct Code', 'Region Name',
       'Media Market', 'Registered Voters', 'Total Voters',
       'Turnout Percentage', 'Sherrod Brown (D)', 'Josh Mandel (R)',
       'Scott Rupert', 'Jim Berns (L)', 'Steve Chabot (R)', 'Jeff Sinnard (D)',
       'Rich Stevenson (G)', 'William Smith (D)', 'Brad Wenstrup (R)',
       'Joyce Beatty (D)', 'Jeff Brown (WI)*', 'Richard Ehrbar (L)',
       'Bob Fitrakis (G)', 'Chris Long (R)', 'Jim Jordan (R)',
       'Chris Kalla (L)', 'Jim Slone (D)', 'Eric Eberly (L)', 'Bob Latta (R)',
       'Angela Zimmann (D)', 'Bill Johnson (R)', 'Charlie Wilson (D)',
       'Bob Gibbs (R)', 'Joyce Healy-Abrams (D)', 'John Boehner (R)',
       'James Condit (WI)*', 'Marcy Kaptur (D)', 'Sean Stipe (L)',
       'Samuel Wurzelbacher (R)', 'David Harlow (L)', 'Sharen Neuhardt (D)',
       'Mike Turner (R)', 'Marcia Fudge (D)', 'Jim Reese (D)',
       'Pat Tiberi (R)', 'Marisha Agana (R)', 'Tim Ryan (D)',
       'Dale Blanchard (D)',

In [6]:
candidates = [i for i in data.columns if i.endswith(')')] #this will remove general information about the election, we only want the details about each canddidate

In [7]:
# keep_cols = [data.columns[0]]

In [8]:
keep_cols = [data.columns[0]] + candidates #add in the column for the county names to the columns for candidates

In [9]:
def trim_party(data,delimiter='('):
    """this function will remove party designations for candidate names with the format used by the OH SoS, unless a different delimiter is called"""
    data_split = [cand.split(delimiter) for cand in data]
    cand_name = [cand[0] for cand in data_split]
    cand_name = [cand.strip() for cand in cand_name]
    return cand_name

In [10]:
data = data[keep_cols] #reduce the dataframe to just the information we need

In [11]:
data.columns = trim_party(data.columns) #apply trim_party to get just candidate names

In [12]:
data.head()

,County Name,Sherrod Brown,Josh Mandel,Jim Berns,Steve Chabot,Jeff Sinnard,Rich Stevenson,William Smith,Brad Wenstrup,Joyce Beatty,...,Marisha Agana,Tim Ryan,Dale Blanchard,David Joyce,David Macko,Elaine Mastromatteo,Pat Lang,Steve Stivers,Jim Renacci,Betty Sutton
0,Total,2.762766e+06,2.435744e+06,9674.000000,201907.000000,131490.000000,6645.000000,137082.000000,194299.000000,201921.000000,...,88120.000000,235492.000000,131638.000000,183660.00000,11536.000000,13038.000000,128188.000000,205277.000000,185167.000000,170604.000000
1,Percentage,5.070107e-01,4.469970e-01,0.027662,0.577346,0.375991,0.019001,0.413669,0.586331,0.682918,...,0.272301,0.727699,0.387316,0.54038,0.033942,0.038362,0.384412,0.615588,0.520467,0.479533
2,Adams,1.850000e+02,4.010000e+02,0.000000,0.000000,0.000000,0.000000,187.000000,405.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Adams,5.100000e+01,1.350000e+02,0.000000,0.000000,0.000000,0.000000,55.000000,140.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Adams,1.170000e+02,1.690000e+02,0.000000,0.000000,0.000000,0.000000,113.000000,170.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
data = data.drop([0,1]).copy()

In [14]:
data = data.groupby('County Name').sum()

In [15]:
data = data.reset_index()

In [16]:
# data

In [24]:
data_2012['SIMPLE_FIRST'] = trim_party(data_2012['CANDIDATE NAME (First)'],' ')

In [25]:
data_2012['(I)'] = data_2012['(I)'].notna().astype(int).copy()
data_2012['CANDIDATE NAME(f)'] = data_2012['SIMPLE_FIRST']+' '+data_2012['CANDIDATE NAME (Last)']

In [26]:
data_2012_simp = data_2012[['STATE ABBREVIATION','D','CANDIDATE NAME(f)','PARTY','(I)']]

In [27]:
def two_party(data):
    """this will reduce the parties to D for Democrat, R for Republican or 'Other' for any 3rd parties
    There is a joke here somewhere about 'this is how the political system actually works'"""
    if data['PARTY'] in ['R', 'D']:
        return data['PARTY']
    else:
        return "OTHER"

In [28]:
data_2012_simp['PARTY'] = data_2012.apply(two_party,axis=1)

C:\Users\joshu\AppData\Local\Temp\ipykernel_11508\340070267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2012_simp['PARTY'] = data_2012.apply(two_party,axis=1)


In [29]:
data_2012_simp

,STATE ABBREVIATION,D,CANDIDATE NAME(f),PARTY,(I)
0,AL,01,Jo Bonner,R,1
1,AL,02,Martha Roby,R,1
2,AL,02,Therese Ford,D,0
3,AL,03,Mike Rogers,R,1
4,AL,03,John Harris,D,0
...,...,...,...,...,...
1480,WY,00,Daniel Cummings,OTHER,0
1481,WY,00,Don Wills,OTHER,0
1482,WY,S,John Barrasso,R,1
1483,WY,S,Tim Chesnut,D,0


In [30]:
house_cand_2012 = data_2012.loc[(data_2012['D']!='S')] #removes all Senate candidates

house_cand_2012.to_csv('house_cand_2012.csv') #save as a csv

In [31]:
county_names = data['County Name']
data_t = data.drop('County Name', axis=1)

In [32]:
data_t=data_t.transpose()

In [33]:
data_t = data_t.drop(['Sherrod Brown','Josh Mandel'], axis=0)

In [34]:
data_t = pd.merge(data_t, data_2012_simp, left_index=True, right_on='CANDIDATE NAME(f)')

In [35]:
# data_2012_simp.loc[data_2012_simp['STATE ABBREVIATION']=='OH']

In [36]:
grouped_data = data_t.groupby(['PARTY','(I)']).sum()

In [37]:
grouped_data = grouped_data.drop(['STATE ABBREVIATION','D','CANDIDATE NAME(f)'],axis=1)

In [38]:
grouped_data

0        1        2        3        4        5        6   \
PARTY (I)                                                                 
D     0    3900.0  14233.0   7260.0  19169.0  17639.0   4298.0  15637.0   
      1       0.0      0.0      0.0      0.0      0.0      0.0      0.0   
OTHER 0       0.0   1594.0      0.0   3533.0      0.0    850.0      0.0   
R     0    6598.0      0.0      0.0  17706.0      0.0      0.0      0.0   
      1       0.0  31072.0  16015.0      0.0   8257.0  17330.0  15182.0   

                7         8       9   ...       78       79      80      81  \
PARTY (I)                             ...                                     
D     0     6612.0       0.0  5815.0  ...  17597.0   6764.0  3534.0  2251.0   
      1        0.0       0.0     0.0  ...      0.0      0.0     0.0     0.0   
OTHER 0        0.0       0.0     0.0  ...      0.0   1400.0   522.0     0.0   
R     0    11660.0       0.0     0.0  ...      0.0      0.0     0.0     0.0   
      1        0.0  121373.0  6858.0  ...  22418.0  15673.0  9464.0  2918.0   

                82       83       84       85       86      87  
PARTY (I)                                                       
D     0    24964.0  11845.0      0.0   5850.0  27744.0  3389.0  
      1        0.0      0.0  19881.0      0.0      0.0     0.0  
OTHER 0     5240.0      0.0      0.0    675.0   2422.0   429.0  
R     0        0.0      0.0      0.0      0.0      0.0     0.0  
      1    76126.0  16652.0  29676.0  10738.0  32128.0  6519.0  

[5 rows x 88 columns]

In [39]:
result_data = grouped_data.apply(lambda x: x.droplevel(1).T.reset_index(drop=True)).reset_index(drop=True)

In [40]:
# result_data['I'] = grouped_data.index.get_level_values('(I)')
# result_data['PARTY'] = grouped_data.index.get_level_values('PARTY')

In [41]:
result_data['index'] = grouped_data.index.get_level_values('PARTY')+grouped_data.index.get_level_values('(I)').astype(str)

In [42]:
result_data

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,index
0,3900.0,14233.0,7260.0,19169.0,17639.0,4298.0,15637.0,6612.0,0.0,5815.0,...,6764.0,3534.0,2251.0,24964.0,11845.0,0.0,5850.0,27744.0,3389.0,D0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19881.0,0.0,0.0,0.0,D1
2,0.0,1594.0,0.0,3533.0,0.0,850.0,0.0,0.0,0.0,0.0,...,1400.0,522.0,0.0,5240.0,0.0,0.0,675.0,2422.0,429.0,OTHER0
3,6598.0,0.0,0.0,17706.0,0.0,0.0,0.0,11660.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,R0
4,0.0,31072.0,16015.0,0.0,8257.0,17330.0,15182.0,0.0,121373.0,6858.0,...,15673.0,9464.0,2918.0,76126.0,16652.0,29676.0,10738.0,32128.0,6519.0,R1


In [43]:
# result_data.set_index('index')

In [44]:
reset_data = result_data.T

In [45]:
reset_data = result_data.T
reset_data.columns = reset_data.iloc[-1]
reset_data = reset_data[:-1]

In [46]:
reset_data.reset_index(inplace=True)

In [47]:
county_names = list(county_names)
# l = ['I','PARTY']

In [48]:
# county_names.append(l[0])
# county_names.append(l[1])

In [49]:
reset_data

index,index,D0,D1,OTHER0,R0,R1
0,0,3900.0,0.0,0.0,6598.0,0.0
1,1,14233.0,0.0,1594.0,0.0,31072.0
2,2,7260.0,0.0,0.0,0.0,16015.0
3,3,19169.0,0.0,3533.0,17706.0,0.0
4,4,17639.0,0.0,0.0,0.0,8257.0
...,...,...,...,...,...,...
83,83,11845.0,0.0,0.0,0.0,16652.0
84,84,0.0,19881.0,0.0,0.0,29676.0
85,85,5850.0,0.0,675.0,0.0,10738.0
86,86,27744.0,0.0,2422.0,0.0,32128.0


In [50]:
reset_data['county_names'] = county_names

In [51]:
# reset_data.set_index('county_names',inplace=True)

In [55]:
reset_data = reset_data.drop('index',axis=1)

In [56]:
OH_2012_House = reset_data[['county_names','D0','D1','OTHER0','R0','R1']]

In [57]:
OH_2012_House

index,county_names,D0,D1,OTHER0,R0,R1
0,Adams,3900.0,0.0,0.0,6598.0,0.0
1,Allen,14233.0,0.0,1594.0,0.0,31072.0
2,Ashland,7260.0,0.0,0.0,0.0,16015.0
3,Ashtabula,19169.0,0.0,3533.0,17706.0,0.0
4,Athens,17639.0,0.0,0.0,0.0,8257.0
...,...,...,...,...,...,...
83,Washington,11845.0,0.0,0.0,0.0,16652.0
84,Wayne,0.0,19881.0,0.0,0.0,29676.0
85,Williams,5850.0,0.0,675.0,0.0,10738.0
86,Wood,27744.0,0.0,2422.0,0.0,32128.0


In [58]:
OH_2012_House.to_csv(r'data/OH_2012_House.csv')

In [59]:
OH_2012_House.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   county_names  88 non-null     object
 1   D0            88 non-null     object
 2   D1            88 non-null     object
 3   OTHER0        88 non-null     object
 4   R0            88 non-null     object
 5   R1            88 non-null     object
dtypes: object(6)
memory usage: 4.3+ KB


In [60]:
OH_2012_IRS = pd.read_csv(r'data/irs_data/irs_count_2012.csv',encoding='latin-1')

In [61]:
OH_2012_IRS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3193 entries, 0 to 3192
Data columns (total 77 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   STATEFIPS   3193 non-null   int64  
 1   STATE       3193 non-null   object 
 2   COUNTYFIPS  3193 non-null   int64  
 3   COUNTYNAME  3193 non-null   object 
 4   AGI_STUB    3193 non-null   int64  
 5   N1          3193 non-null   float64
 6   MARS1       3193 non-null   float64
 7   MARS2       3193 non-null   float64
 8   MARS4       3193 non-null   float64
 9   PREP        3193 non-null   float64
 10  N2          3193 non-null   float64
 11  NUMDEP      3193 non-null   float64
 12  A00100      3193 non-null   float64
 13  N00200      3193 non-null   float64
 14  A00200      3193 non-null   float64
 15  N00300      3193 non-null   float64
 16  A00300      3193 non-null   float64
 17  N00600      3193 non-null   float64
 18  A00600      3193 non-null   float64
 19  N00650      3193 non-null  

In [62]:
OH_2012_IRS.columns

Index(['STATEFIPS', 'STATE', 'COUNTYFIPS', 'COUNTYNAME', 'AGI_STUB', 'N1',
       'MARS1', 'MARS2', 'MARS4', 'PREP', 'N2', 'NUMDEP', 'A00100', 'N00200',
       'A00200', 'N00300', 'A00300', 'N00600', 'A00600', 'N00650', 'A00650',
       'N00900', 'A00900', 'SCHF', 'N01000', 'A01000', 'N01400', 'A01400',
       'N01700', 'A01700', 'N02300', 'A02300', 'N02500', 'A02500', 'N03300',
       'A03300', 'N04470', 'A00101', 'A04470', 'N18425', 'A18425', 'N18450',
       'A18450', 'N18500', 'A18500', 'N18300', 'A18300', 'N19300', 'A19300',
       'N19700', 'A19700', 'N04800', 'A04800', 'N09600', 'A09600', 'N07100',
       'A07100', 'N07180', 'A07180', 'N07220', 'A07220', 'N07260', 'A07260',
       'N59660', 'A59660', 'N59720', 'A59720', 'N11070', 'A11070', 'N06500',
       'A06500', 'N10300', 'A10300', 'N11901', 'A11901', 'N11902', 'A11902'],
      dtype='object')

In [63]:
OH_2012_IRS = OH_2012_IRS.loc[OH_2012_IRS['STATE']=='OH']

In [64]:
OH_2012_IRS

,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,AGI_STUB,N1,MARS1,MARS2,MARS4,PREP,...,N11070,A11070,N06500,A06500,N10300,A10300,N11901,A11901,N11902,A11902
2077,39,OH,0,Ohio,0,5506020.0,2512780.0,1909410.0,715800.0,2788300.0,...,648480.0,829786.0,3805570.0,35320490.0,4078270.0,37000681.0,848210.0,3747223.0,4394820.0,11124637.0
2078,39,OH,1,Adams County,0,11060.0,4050.0,4890.0,1460.0,7550.0,...,1930.0,2462.0,6730.0,32540.0,7770.0,35947.0,1290.0,3779.0,9250.0,25728.0
2079,39,OH,3,Allen County,0,47820.0,21000.0,17080.0,6510.0,25580.0,...,6390.0,7906.0,31820.0,246179.0,34110.0,259089.0,6670.0,33018.0,39260.0,100692.0
2080,39,OH,5,Ashland County,0,24290.0,10260.0,9850.0,2170.0,12960.0,...,2530.0,3325.0,16870.0,102753.0,18000.0,109401.0,3550.0,14853.0,19430.0,45037.0
2081,39,OH,7,Ashtabula County,0,44960.0,19690.0,16730.0,5880.0,23350.0,...,5950.0,7616.0,29040.0,162592.0,31450.0,171646.0,5730.0,18210.0,36910.0,90020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2161,39,OH,167,Washington County,0,28840.0,12420.0,12030.0,2800.0,14730.0,...,3150.0,3828.0,19950.0,152982.0,21250.0,161283.0,4150.0,26225.0,23360.0,57419.0
2162,39,OH,169,Wayne County,0,53570.0,22960.0,21940.0,4500.0,31160.0,...,5760.0,8215.0,36990.0,254406.0,39630.0,271093.0,8390.0,32460.0,41980.0,97016.0
2163,39,OH,171,Williams County,0,18420.0,7930.0,7070.0,1990.0,11130.0,...,2200.0,2799.0,12550.0,72130.0,13420.0,77548.0,2580.0,11478.0,14910.0,36824.0
2164,39,OH,173,Wood County,0,59860.0,27120.0,23000.0,5360.0,30450.0,...,4600.0,5794.0,44160.0,390218.0,46170.0,410398.0,9670.0,43328.0,47270.0,114491.0


In [65]:
def remove_keyword(data, keyword):
    trunk = []

    for i in data:
        if keyword in i:
            # Remove the keyword, strip, and add to the new list
            mod = i.replace(keyword, '').strip()
        else:
            # If the keyword is not present, keep the original entry
            mod = i

        trunk.append(mod)

    return trunk


In [66]:
OH_2012_IRS['COUNTYNAME']=remove_keyword(OH_2012_IRS['COUNTYNAME'],'County')

In [67]:
OH_2012_IRS

,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,AGI_STUB,N1,MARS1,MARS2,MARS4,PREP,...,N11070,A11070,N06500,A06500,N10300,A10300,N11901,A11901,N11902,A11902
2077,39,OH,0,Ohio,0,5506020.0,2512780.0,1909410.0,715800.0,2788300.0,...,648480.0,829786.0,3805570.0,35320490.0,4078270.0,37000681.0,848210.0,3747223.0,4394820.0,11124637.0
2078,39,OH,1,Adams,0,11060.0,4050.0,4890.0,1460.0,7550.0,...,1930.0,2462.0,6730.0,32540.0,7770.0,35947.0,1290.0,3779.0,9250.0,25728.0
2079,39,OH,3,Allen,0,47820.0,21000.0,17080.0,6510.0,25580.0,...,6390.0,7906.0,31820.0,246179.0,34110.0,259089.0,6670.0,33018.0,39260.0,100692.0
2080,39,OH,5,Ashland,0,24290.0,10260.0,9850.0,2170.0,12960.0,...,2530.0,3325.0,16870.0,102753.0,18000.0,109401.0,3550.0,14853.0,19430.0,45037.0
2081,39,OH,7,Ashtabula,0,44960.0,19690.0,16730.0,5880.0,23350.0,...,5950.0,7616.0,29040.0,162592.0,31450.0,171646.0,5730.0,18210.0,36910.0,90020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2161,39,OH,167,Washington,0,28840.0,12420.0,12030.0,2800.0,14730.0,...,3150.0,3828.0,19950.0,152982.0,21250.0,161283.0,4150.0,26225.0,23360.0,57419.0
2162,39,OH,169,Wayne,0,53570.0,22960.0,21940.0,4500.0,31160.0,...,5760.0,8215.0,36990.0,254406.0,39630.0,271093.0,8390.0,32460.0,41980.0,97016.0
2163,39,OH,171,Williams,0,18420.0,7930.0,7070.0,1990.0,11130.0,...,2200.0,2799.0,12550.0,72130.0,13420.0,77548.0,2580.0,11478.0,14910.0,36824.0
2164,39,OH,173,Wood,0,59860.0,27120.0,23000.0,5360.0,30450.0,...,4600.0,5794.0,44160.0,390218.0,46170.0,410398.0,9670.0,43328.0,47270.0,114491.0


In [68]:
OH_2012_IRS = OH_2012_IRS.drop(['STATEFIPS','AGI_STUB','COUNTYFIPS'],axis=1)

In [69]:
OH_2012_merged = pd.merge(OH_2012_House,OH_2012_IRS, how='left',left_on='county_names', right_on='COUNTYNAME')

In [70]:
OH_2012_merged

,county_names,D0,D1,OTHER0,R0,R1,STATE,COUNTYFIPS,COUNTYNAME,N1,...,N11070,A11070,N06500,A06500,N10300,A10300,N11901,A11901,N11902,A11902
0,Adams,3900.0,0.0,0.0,6598.0,0.0,OH,1,Adams,11060.0,...,1930.0,2462.0,6730.0,32540.0,7770.0,35947.0,1290.0,3779.0,9250.0,25728.0
1,Allen,14233.0,0.0,1594.0,0.0,31072.0,OH,3,Allen,47820.0,...,6390.0,7906.0,31820.0,246179.0,34110.0,259089.0,6670.0,33018.0,39260.0,100692.0
2,Ashland,7260.0,0.0,0.0,0.0,16015.0,OH,5,Ashland,24290.0,...,2530.0,3325.0,16870.0,102753.0,18000.0,109401.0,3550.0,14853.0,19430.0,45037.0
3,Ashtabula,19169.0,0.0,3533.0,17706.0,0.0,OH,7,Ashtabula,44960.0,...,5950.0,7616.0,29040.0,162592.0,31450.0,171646.0,5730.0,18210.0,36910.0,90020.0
4,Athens,17639.0,0.0,0.0,0.0,8257.0,OH,9,Athens,21670.0,...,2420.0,2929.0,14370.0,99427.0,15550.0,105462.0,2960.0,11463.0,17740.0,41462.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Washington,11845.0,0.0,0.0,0.0,16652.0,OH,167,Washington,28840.0,...,3150.0,3828.0,19950.0,152982.0,21250.0,161283.0,4150.0,26225.0,23360.0,57419.0
84,Wayne,0.0,19881.0,0.0,0.0,29676.0,OH,169,Wayne,53570.0,...,5760.0,8215.0,36990.0,254406.0,39630.0,271093.0,8390.0,32460.0,41980.0,97016.0
85,Williams,5850.0,0.0,675.0,0.0,10738.0,OH,171,Williams,18420.0,...,2200.0,2799.0,12550.0,72130.0,13420.0,77548.0,2580.0,11478.0,14910.0,36824.0
86,Wood,27744.0,0.0,2422.0,0.0,32128.0,OH,173,Wood,59860.0,...,4600.0,5794.0,44160.0,390218.0,46170.0,410398.0,9670.0,43328.0,47270.0,114491.0


In [73]:
dnmatch = OH_2012_merged[OH_2012_merged['county_names'] != OH_2012_merged['COUNTYNAME']]

In [74]:
dnmatch

,county_names,D0,D1,OTHER0,R0,R1,STATE,COUNTYFIPS,COUNTYNAME,N1,...,N11070,A11070,N06500,A06500,N10300,A10300,N11901,A11901,N11902,A11902
